In [ ]:
import sympy as sp
import sys

sys.path.append("../")
import pathlib as pl
from SymEigen import *
from sympy import symbols
from project_dir import backend_source_dir

Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__host__ __device__")

def compute_J_point(xbar):
    J = sp.Matrix.zeros(3, 12)
    J[0:3, 0:3] = sp.eye(3)
    J[0, 3:6] = xbar.T
    J[1, 6:9] = xbar.T
    J[2, 9:12] = xbar.T
    return J


def compute_J_vec(xbar):
    J = sp.Matrix.zeros(3, 12)
    # Remove the translation part
    # we only need the rotation / scaling part for vector transformation
    # J[0:3, 0:3] = sp.eye(3)
    J[0, 3:6] = xbar.T
    J[1, 6:9] = xbar.T
    J[2, 9:12] = xbar.T
    return J

kappa  = Eigen.Scalar("kappa")

In [ ]:
Q = Eigen.Vector("Q", 9)
C0 = Q[0:3,0]
D0 = Q[3:6,0]
C1 = Q[6:9,0]
R = (C1 - C0).cross(D0 - C0)
E = 1 / 2 * kappa[0] * R.T * R
dEdQ = VecDiff(E, Q)
print(dEdQ.shape)
ddEddQ = VecDiff(dEdQ, Q)
print(ddEddQ.shape)
ClE = Gen.Closure(kappa,Q)

In [ ]:
C0_bar = Eigen.Vector("C0_bar", 3)
D0_bar = Eigen.Vector("D0_bar", 3)
C1_bar = Eigen.Vector("C1_bar", 3)
J = Matrix.zeros(9,24)
J[0:3, 0:12] = compute_J_point(C0_bar)
J[3:6, 0:12] = compute_J_point(D0_bar)
J[6:9, 12:24] = compute_J_point(C1_bar)
Grad = J.T * dEdQ
print(Grad.shape)
ClG = Gen.Closure(kappa, Q, C0_bar,D0_bar, C1_bar)
H9x9 = Eigen.Matrix("ddEddQ", 9, 9)
Hess = J.T * H9x9 * J
print(Hess.shape)
ClH = Gen.Closure(kappa, C0_bar,D0_bar, C1_bar, H9x9)

In [ ]:
# X is two vector
X = Eigen.Vector("X", 6)
X0 = X[0:3, 0]
X1 = X[3:6, 0]
EX = 1/ 2 * kappa[0] * (X1 - X0).T * (X1 - X0)
dEdX = VecDiff(EX, X)
print(dEdX.shape)
ddEddX = VecDiff(dEdX, X)
print(ddEddX.shape)
ClX = Gen.Closure(kappa, X)
# grad and hessian for normal and tangent vectors
X0_bar = Eigen.Vector("X0_bar", 3)
X1_bar = Eigen.Vector("X1_bar", 3)

J = Matrix.zeros(6, 24)
J[0:3, 0:12] = compute_J_vec(X0_bar)
J[3:6, 12:24] = compute_J_vec(X1_bar)
GradX = J.T * dEdX
print(GradX.shape)
ClXG = Gen.Closure(kappa, X,X0_bar, X1_bar)
H6x6 = Eigen.Matrix("ddEddX", 6 , 6)
HessX = J.T * H6x6 * J
print(HessX.shape)
ClXH = Gen.Closure(kappa, X0_bar, X1_bar, H6x6)

In [ ]:
s = f"""// Affine Body Prismatic Joint
{ClE("dEdQ", dEdQ)}
{ClE("ddEddQ", ddEddQ)}
{ClG("GradE", Grad)}
{ClH("HessE", Hess)}
{ClX("ddEddX", ddEddX)}
{ClXG("GradX", GradX)}
{ClXH("HessX", HessX)}
"""
print(s)

f = open(
    backend_source_dir("cuda")
    / "affine_body/constitutions/sym/affine_body_prismatic_joint.inl",
    "w",
)
f.write(s)
f.close()
